# Content Based Books Recommender System using cosine similarity

In [1]:
## import necessary libraries
import numpy as np
import pandas as pd

import re
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
## upload books dataset
books = pd.read_csv('books.csv')
books.head()

id  book_id  best_book_id  work_id  books_count       isbn        isbn13  \
0   1  2767052       2767052  2792775          272  439023483  9.780439e+12   
1   2        3             3  4640799          491  439554934  9.780440e+12   
2   3    41865         41865  3212258          226  316015849  9.780316e+12   
3   4     2657          2657  3275794          487   61120081  9.780061e+12   
4   5     4671          4671   245494         1356  743273567  9.780743e+12   

                       authors  original_publication_year  \
0              Suzanne Collins                     2008.0   
1  J.K. Rowling, Mary GrandPré                     1997.0   
2              Stephenie Meyer                     2005.0   
3                   Harper Lee                     1960.0   
4          F. Scott Fitzgerald                     1925.0   

                             original_title  ... ratings_count  \
0                          The Hunger Games  ...       4780653   
1  Harry Potter and the Philosopher's Stone  ...       4602479   
2                                  Twilight  ...       3866839   
3                     To Kill a Mockingbird  ...       3198671   
4                          The Great Gatsby  ...       2683664   

  work_ratings_count  work_text_reviews_count  ratings_1  ratings_2  \
0            4942365                   155254      66715     127936   
1            4800065                    75867      75504     101676   
2            3916824                    95009     456191     436802   
3            3340896                    72586      60427     117415   
4            2773745                    51992      86236     197621   

   ratings_3  ratings_4  ratings_5  \
0     560092    1481305    2706317   
1     455024    1156318    3011543   
2     793319     875073    1355439   
3     446835    1001952    1714267   
4     606158     936012     947718   

                                           image_url  \
0  https://images.gr-assets.com/books/1447303603m...   
1  https://images.gr-assets.com/books/1474154022m...   
2  https://images.gr-assets.com/books/1361039443m...   
3  https://images.gr-assets.com/books/1361975680m...   
4  https://images.gr-assets.com/books/1490528560m...   

                                     small_image_url  
0  https://images.gr-assets.com/books/1447303603s...  
1  https://images.gr-assets.com/books/1474154022s...  
2  https://images.gr-assets.com/books/1361039443s...  
3  https://images.gr-assets.com/books/1361975680s...  
4  https://images.gr-assets.com/books/1490528560s...  

[5 rows x 23 columns]

In [43]:
books['small_image_url'].head()

0    https://images.gr-assets.com/books/1447303603s...
1    https://images.gr-assets.com/books/1474154022s...
2    https://images.gr-assets.com/books/1361039443s...
3    https://images.gr-assets.com/books/1361975680s...
4    https://images.gr-assets.com/books/1490528560s...
Name: small_image_url, dtype: object

In [3]:
## upload ratings dataset

ratings = pd.read_csv('ratings.csv')
ratings.head()

book_id  user_id  rating
0        1      314       5
1        1      439       3
2        1      588       5
3        1     1169       4
4        1     1185       4

In [4]:
books.shape

(10000, 23)

In [5]:
ratings.shape

(981756, 3)

In [6]:
len(set(ratings['user_id']))

53424

In [7]:
len(set(ratings['book_id']))

10000

In [8]:
books['id'].nunique()

10000

In [9]:
books.describe()

id       book_id  best_book_id       work_id   books_count  \
count  10000.00000  1.000000e+04  1.000000e+04  1.000000e+04  10000.000000   
mean    5000.50000  5.264697e+06  5.471214e+06  8.646183e+06     75.712700   
std     2886.89568  7.575462e+06  7.827330e+06  1.175106e+07    170.470728   
min        1.00000  1.000000e+00  1.000000e+00  8.700000e+01      1.000000   
25%     2500.75000  4.627575e+04  4.791175e+04  1.008841e+06     23.000000   
50%     5000.50000  3.949655e+05  4.251235e+05  2.719524e+06     40.000000   
75%     7500.25000  9.382225e+06  9.636112e+06  1.451775e+07     67.000000   
max    10000.00000  3.328864e+07  3.553423e+07  5.639960e+07   3455.000000   

             isbn13  original_publication_year  average_rating  ratings_count  \
count  9.415000e+03                9979.000000    10000.000000   1.000000e+04   
mean   9.755044e+12                1981.987674        4.002191   5.400124e+04   
std    4.428619e+11                 152.576665        0.254427   1.573700e+05   
min    1.951703e+08               -1750.000000        2.470000   2.716000e+03   
25%    9.780316e+12                1990.000000        3.850000   1.356875e+04   
50%    9.780452e+12                2004.000000        4.020000   2.115550e+04   
75%    9.780831e+12                2011.000000        4.180000   4.105350e+04   
max    9.790008e+12                2017.000000        4.820000   4.780653e+06   

       work_ratings_count  work_text_reviews_count      ratings_1  \
count        1.000000e+04             10000.000000   10000.000000   
mean         5.968732e+04              2919.955300    1345.040600   
std          1.678038e+05              6124.378132    6635.626263   
min          5.510000e+03                 3.000000      11.000000   
25%          1.543875e+04               694.000000     196.000000   
50%          2.383250e+04              1402.000000     391.000000   
75%          4.591500e+04              2744.250000     885.000000   
max          4.942365e+06            155254.000000  456191.000000   

           ratings_2      ratings_3     ratings_4     ratings_5  
count   10000.000000   10000.000000  1.000000e+04  1.000000e+04  
mean     3110.885000   11475.893800  1.996570e+04  2.378981e+04  
std      9717.123578   28546.449183  5.144736e+04  7.976889e+04  
min        30.000000     323.000000  7.500000e+02  7.540000e+02  
25%       656.000000    3112.000000  5.405750e+03  5.334000e+03  
50%      1163.000000    4894.000000  8.269500e+03  8.836000e+03  
75%      2353.250000    9287.000000  1.602350e+04  1.730450e+04  
max    436802.000000  793319.000000  1.481305e+06  3.011543e+06

In [10]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         10000 non-null  int64  
 1   book_id                    10000 non-null  int64  
 2   best_book_id               10000 non-null  int64  
 3   work_id                    10000 non-null  int64  
 4   books_count                10000 non-null  int64  
 5   isbn                       9300 non-null   object 
 6   isbn13                     9415 non-null   float64
 7   authors                    10000 non-null  object 
 8   original_publication_year  9979 non-null   float64
 9   original_title             9415 non-null   object 
 10  title                      10000 non-null  object 
 11  language_code              8916 non-null   object 
 12  average_rating             10000 non-null  float64
 13  ratings_count              10000 non-null  int6

## Data Cleaning

In [11]:
books = books.dropna()

books.drop_duplicates(subset='original_title', keep='first', inplace=True)

In [12]:
books.head(3)

id  book_id  best_book_id  work_id  books_count       isbn        isbn13  \
0   1  2767052       2767052  2792775          272  439023483  9.780439e+12   
1   2        3             3  4640799          491  439554934  9.780440e+12   
2   3    41865         41865  3212258          226  316015849  9.780316e+12   

                       authors  original_publication_year  \
0              Suzanne Collins                     2008.0   
1  J.K. Rowling, Mary GrandPré                     1997.0   
2              Stephenie Meyer                     2005.0   

                             original_title  ... ratings_count  \
0                          The Hunger Games  ...       4780653   
1  Harry Potter and the Philosopher's Stone  ...       4602479   
2                                  Twilight  ...       3866839   

  work_ratings_count  work_text_reviews_count  ratings_1  ratings_2  \
0            4942365                   155254      66715     127936   
1            4800065                    75867      75504     101676   
2            3916824                    95009     456191     436802   

   ratings_3  ratings_4  ratings_5  \
0     560092    1481305    2706317   
1     455024    1156318    3011543   
2     793319     875073    1355439   

                                           image_url  \
0  https://images.gr-assets.com/books/1447303603m...   
1  https://images.gr-assets.com/books/1474154022m...   
2  https://images.gr-assets.com/books/1361039443m...   

                                     small_image_url  
0  https://images.gr-assets.com/books/1447303603s...  
1  https://images.gr-assets.com/books/1474154022s...  
2  https://images.gr-assets.com/books/1361039443s...  

[3 rows x 23 columns]

In [13]:
## select records if 'average_rating'>3 and 'ratings_count'>13568

books_filter = pd.DataFrame(books[(books['average_rating']>3) & (books['ratings_count']>13568)])
books_filter.head()

id  book_id  best_book_id  work_id  books_count       isbn        isbn13  \
0   1  2767052       2767052  2792775          272  439023483  9.780439e+12   
1   2        3             3  4640799          491  439554934  9.780440e+12   
2   3    41865         41865  3212258          226  316015849  9.780316e+12   
3   4     2657          2657  3275794          487   61120081  9.780061e+12   
4   5     4671          4671   245494         1356  743273567  9.780743e+12   

                       authors  original_publication_year  \
0              Suzanne Collins                     2008.0   
1  J.K. Rowling, Mary GrandPré                     1997.0   
2              Stephenie Meyer                     2005.0   
3                   Harper Lee                     1960.0   
4          F. Scott Fitzgerald                     1925.0   

                             original_title  ... ratings_count  \
0                          The Hunger Games  ...       4780653   
1  Harry Potter and the Philosopher's Stone  ...       4602479   
2                                  Twilight  ...       3866839   
3                     To Kill a Mockingbird  ...       3198671   
4                          The Great Gatsby  ...       2683664   

  work_ratings_count  work_text_reviews_count  ratings_1  ratings_2  \
0            4942365                   155254      66715     127936   
1            4800065                    75867      75504     101676   
2            3916824                    95009     456191     436802   
3            3340896                    72586      60427     117415   
4            2773745                    51992      86236     197621   

   ratings_3  ratings_4  ratings_5  \
0     560092    1481305    2706317   
1     455024    1156318    3011543   
2     793319     875073    1355439   
3     446835    1001952    1714267   
4     606158     936012     947718   

                                           image_url  \
0  https://images.gr-assets.com/books/1447303603m...   
1  https://images.gr-assets.com/books/1474154022m...   
2  https://images.gr-assets.com/books/1361039443m...   
3  https://images.gr-assets.com/books/1361975680m...   
4  https://images.gr-assets.com/books/1490528560m...   

                                     small_image_url  
0  https://images.gr-assets.com/books/1447303603s...  
1  https://images.gr-assets.com/books/1474154022s...  
2  https://images.gr-assets.com/books/1361039443s...  
3  https://images.gr-assets.com/books/1361975680s...  
4  https://images.gr-assets.com/books/1490528560s...  

[5 rows x 23 columns]

In [16]:
## fetch required columns

content_data = books_filter[['original_title','authors','average_rating']]

## convert data type into string
content_data = content_data.astype(str)

## add 'original_title' and 'authors' content
content_data['content'] = content_data['original_title'] + ' ' + content_data['authors'] + ' ' + content_data['average_rating']

In [17]:
content_data

original_title                      authors  \
0                             The Hunger Games              Suzanne Collins   
1     Harry Potter and the Philosopher's Stone  J.K. Rowling, Mary GrandPré   
2                                     Twilight              Stephenie Meyer   
3                        To Kill a Mockingbird                   Harper Lee   
4                             The Great Gatsby          F. Scott Fitzgerald   
...                                        ...                          ...   
9891                        Salvation in Death                    J.D. Robb   
9923               Blue Noon (Midnighters, #3)             Scott Westerfeld   
9949                               Ride Steady               Kristen Ashley   
9954                                Wild Cards               Simone Elkeles   
9995                                Bayou Moon                Ilona Andrews   

     average_rating                                            content  
0              4.34              The Hunger Games Suzanne Collins 4.34  
1              4.44  Harry Potter and the Philosopher's Stone J.K. ...  
2              3.57                      Twilight Stephenie Meyer 3.57  
3              4.25              To Kill a Mockingbird Harper Lee 4.25  
4              3.89          The Great Gatsby F. Scott Fitzgerald 3.89  
...             ...                                                ...  
9891           4.26                  Salvation in Death J.D. Robb 4.26  
9923           3.85  Blue Noon (Midnighters, #3) Scott Westerfeld 3.85  
9949           4.45                    Ride Steady Kristen Ashley 4.45  
9954           3.95                     Wild Cards Simone Elkeles 3.95  
9995           4.09                      Bayou Moon Ilona Andrews 4.09  

[6037 rows x 4 columns]

In [18]:
type(content_data)

pandas.core.frame.DataFrame

In [19]:
content_data.head()

original_title                      authors  \
0                          The Hunger Games              Suzanne Collins   
1  Harry Potter and the Philosopher's Stone  J.K. Rowling, Mary GrandPré   
2                                  Twilight              Stephenie Meyer   
3                     To Kill a Mockingbird                   Harper Lee   
4                          The Great Gatsby          F. Scott Fitzgerald   

  average_rating                                            content  
0           4.34              The Hunger Games Suzanne Collins 4.34  
1           4.44  Harry Potter and the Philosopher's Stone J.K. ...  
2           3.57                      Twilight Stephenie Meyer 3.57  
3           4.25              To Kill a Mockingbird Harper Lee 4.25  
4           3.89          The Great Gatsby F. Scott Fitzgerald 3.89

In [20]:
## convert string into lower case 
content_data['content'] = content_data['content'].apply(lambda x:x.lower())

## Count Vectorizer : text to vector

In [21]:
## Stemming

import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [22]:
def stem(text):
    y =[]
    
    for i in text.split(): ## convert list into string
        y.append(ps.stem(i))
        
    return " ".join(y)  ## convert string into list
        

In [23]:
content_data['content'] = content_data['content'].apply(stem)

In [24]:
## import count vectorizer library

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [25]:
vectors =cv.fit_transform(content_data['content']).toarray()

In [26]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [27]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [28]:
## length of features
cv.get_feature_names()

C:\Users\Arif Khan\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '100',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '1942',
 '1944',
 '1962',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '365',
 '37',
 '38',
 '39',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 'aaron',
 'aaronovitch',
 'abbey',
 'abbi',
 'abbott',
 'abel',
 'abercrombi',
 'abraham',
 'abroad',
 'absalom',
 'absolut',
 'academ',
 'academi',
 'accept',
 'accord',
 'account',
 'achiev',
 'ackroyd',
 'act',
 'action',
 'adam',
 'addict',
 'addison',
 'adelin',
 'adichi',
 'adlard',
 'adler',
 'adornetto'

In [29]:
len(cv.get_feature_names())

5000

## Cosine similarity

In [30]:
from sklearn.metrics.pairwise import cosine_similarity

In [31]:
similarity = cosine_similarity(vectors)

In [32]:
similarity[0]

array([1., 0., 0., ..., 0., 0., 0.])

In [33]:
similarity[1]

array([0., 1., 0., ..., 0., 0., 0.])

## Make function for Recommendation 

In [42]:
def recommend(book):
    book_index = content_data[content_data['original_title'] == book].index[0]
    distances = similarity[book_index]
    books_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    
    for i in books_list:
        print(content_data.iloc[i[0]].original_title)
        

In [36]:
recommend('Gregor the Overlander') 

Flash Boys: A Wall Street Revolt
The Blind Side: Evolution of a Game
The Four Loves
Bartleby, the Scrivener: A Story of Wall Street
The Slap


In [37]:
content_data.head()

original_title                      authors  \
0                          The Hunger Games              Suzanne Collins   
1  Harry Potter and the Philosopher's Stone  J.K. Rowling, Mary GrandPré   
2                                  Twilight              Stephenie Meyer   
3                     To Kill a Mockingbird                   Harper Lee   
4                          The Great Gatsby          F. Scott Fitzgerald   

  average_rating                                            content  
0           4.34                 the hunger game suzann collin 4.34  
1           4.44  harri potter and the philosopher' stone j.k. r...  
2           3.57                       twilight stepheni meyer 3.57  
3           4.25              to kill a mockingbird harper lee 4.25  
4           3.89          the great gatsbi f. scott fitzgerald 3.89

In [38]:
content_data[content_data['original_title']=='The Slap']

original_title            authors average_rating  \
4783       The Slap  Christos Tsiolkas           3.16   

                            content  
4783  the slap christo tsiolka 3.16

In [39]:
import pickle
pickle.dump(content_data,open('book_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

In [ ]:
from platform import python_version
print(python_version())